# 🧪 ResNet Fine-Tuning 실험

ResNet 구조의 하이퍼파라미터를 조정하며 성능 개선을 시도한 실험입니다.
- Dropout 비율 조절
- Epoch, batch size 튜닝
- 모델 성능 비교 결과 포함


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install split-folders

In [ ]:
!pip install split-folders

In [ ]:
%load_ext tensorboard


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization
import matplotlib.pyplot as plt
import datetime
import os
import cv2
import numpy as np
import pickle

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import splitfolders

import shutil

In [ ]:
!unzip -q /content/drive/MyDrive/resize.zip

In [ ]:

folder_path = '/content/resize'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'normal': ['/content/resize/normal/d7c3d3207615088285162f2c4b7ccd477da80c8fca7fa2615d5abe6cbac3ee38_40_20201206182753_010_008.jpg',
  '/content/resize/normal/e7d03573a4ef0f455e412779fe03127267f8674ef9e99e851cba2a0d75a134c4_30_20201205122800_006_006.jpg',
  '/content/resize/normal/e47d351bb4db04b5e3d51e39fbda0aeb5f7ab0dc1effb2f840e64ac1aa43dea9_30_20210209020807_006_002.jpg',
  '/content/resize/normal/ddce63cf979bcffe75e72eb7ab4eb802fe98e253e7cef396fce80a4fb8831c8b_20_20210204171437_006_004.jpg',
  '/content/resize/normal/f5da5b94de34a3a4b2579b4e2fe296f47eaf1dd6b6d11f91975b12983d5c181b_30_20201202180834_010_003.jpg',
  '/content/resize/normal/ef81443fa34c7641579df603ba14dc6466a2b7074446c5f2078c0594f125a8c8_20_20201207005916_006_011.jpg',
  '/content/resize/normal/e10c6634bc727f46e79ccde300fcf949df69a3634659444e098016d7323871dc_20_20201207011915_010_019.jpg',
  '/content/resize/normal/e47d351bb4db04b5e3d51e39fbda0aeb5f7ab0dc1effb2f840e64ac1aa43dea9_30_20210209020807_006_017.jpg',
  '/co

In [ ]:
os.mkdir('/content/preprss')
for label in os.listdir(folder_path):
    dir_path = '/content/preprss/'+ label
    os.mkdir(dir_path)

In [ ]:
def change_img(img_path, target_size=224):
    img = cv2.imread(img_path)

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_img = np.expand_dims(gray_img,axis=-1)
    img_tensor = tf.convert_to_tensor(gray_img, dtype=tf.float32)
    # img_tensor = tf.image.central_crop(img_tensor, central_fraction=0.5)
    img_cropped = img_tensor.numpy()

    w, h = img_cropped.shape[1], img_cropped.shape[0]
    dw = (target_size-w)/2 # img_size와 w의 차이
    dh = (target_size-h)/2 # img_size와 h의 차이
    M = np.float32([[1,0,dw], [0,1,dh]])  #(2*3 이차원 행렬)
    img_re = cv2.warpAffine(img_cropped, M, (224, 224)) #이동변환
    filename = img_path.split("/")[-1]
    cv2.imwrite(f'/content/preprss/{label}/{filename}', img_re) # 경로 추가

In [ ]:
dataset = {}
for label, img_paths in dataset.items():  #'/content/resize' -> content/preprss
    for img_path in img_paths:
      change_img(img_path)

In [ ]:
splitfolders.ratio('/content/preprss', output='dataset', seed=77, ratio=(0.6, 0.2, 0.2))

NameError: name 'splitfolders' is not defined

In [ ]:
folder_path = '/content/dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

{'normal': ['/content/dataset/train/normal/d7c3d3207615088285162f2c4b7ccd477da80c8fca7fa2615d5abe6cbac3ee38_40_20201206182753_010_008.jpg',
  '/content/dataset/train/normal/e10c6634bc727f46e79ccde300fcf949df69a3634659444e098016d7323871dc_20_20201207011915_010_019.jpg',
  '/content/dataset/train/normal/e47d351bb4db04b5e3d51e39fbda0aeb5f7ab0dc1effb2f840e64ac1aa43dea9_30_20210209020807_006_017.jpg',
  '/content/dataset/train/normal/df0625b55a97911ade446f291f27a6c53f561e59e6f4230928a696cce40e9946_50_20210208141956_010_001.jpg',
  '/content/dataset/train/normal/e9856a72809e0ed4225b4352f549d76ed2b9cd79f7e82d4f493e799c60d10952_20_20201205174309_010_013.jpg',
  '/content/dataset/train/normal/e9856a72809e0ed4225b4352f549d76ed2b9cd79f7e82d4f493e799c60d10952_20_20201205174309_010_010.jpg',
  '/content/dataset/train/normal/e4a3ae1139f2ef5d7d850dd39dd7cb7d5a71aceaaf6fd08065b94b98fd981934_20_20201207191602_010_015.jpg',
  '/content/dataset/train/normal/e47d351bb4db04b5e3d51e39fbda0aeb5f7ab0dc1effb2f

In [ ]:

x_train = []

for label, img_paths in dataset.items():
    for img_path in img_paths:
        image = cv2.imread(img_path) # img를 array 형태로 변경

        x_train.append(image)

x_train = np.array(x_train)

In [ ]:
n_train = x_train.shape[0]

In [ ]:
# compute_mean for zero-centering
compute_mean = lambda imgs : np.mean(imgs, axis=0)

In [ ]:
mean_img = compute_mean(x_train)
mean_img.shape

(224, 224, 3)

In [ ]:
# zero centering function
sub_mean_img = lambda image : image.astype('int8') - mean_img.astype('int8')

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # vertical_flip=True,
    preprocessing_function=sub_mean_img
)

val_datagen = ImageDataGenerator(
    preprocessing_function=sub_mean_img)

test_datagen = ImageDataGenerator(
    preprocessing_function=sub_mean_img)

In [ ]:
batch_size = 200

In [ ]:
train_generator = train_datagen.flow_from_directory('/content/dataset/train',batch_size=batch_size, target_size=(224, 224))
val_generator = val_datagen.flow_from_directory('/content/dataset/val',batch_size=batch_size, target_size=(224, 224))
test_generator = test_datagen.flow_from_directory('/content/dataset/test',batch_size=batch_size, target_size=(224, 224))

Found 4199 images belonging to 7 classes.
Found 1399 images belonging to 7 classes.
Found 1401 images belonging to 7 classes.


In [ ]:
base_model = ResNet50(include_top=False, input_shape = (224,224 ,3), weights = 'imagenet')
base_model.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def create_resnet_model(n_hidden_node, dropout_prob):

    base_model = ResNet50(include_top=False, input_shape = (224, 224 ,3), weights = 'imagenet')
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))

    x = base_model(inputs, training=False)
    x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
    x = tf.keras.layers.Dense(n_hidden_node, activation='relu')(x)
    x= tf.keras.layers.Dropout(dropout_prob)(x)
    outputs = tf.keras.layers.Dense(7, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    return model

In [ ]:

batch_size

150

In [ ]:
model = create_resnet_model(256, 0.5)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
current_time01 = datetime.datetime.now().strftime("%Y%m%d")
current_time02 = datetime.datetime.now().strftime("%H%M%S")

current_time = current_time01
model_name = 'resnet_flip'

log_dir = f"logs/fit/{current_time}_{model_name}"

In [ ]:

checkpoint_path = "training_1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
callbacks = [
    # tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=0),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=True),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    ]

In [ ]:
# 모델 fitting
model.fit(train_generator, batch_size=batch_size, validation_data=val_generator, epochs=10, callbacks = callbacks)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


28/28 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.1401 - loss: 55.6890 - val_accuracy: 0.1430 - val_loss: 1.9460 - learning_rate: 0.0100
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.1373 - loss: 1.9465 - val_accuracy: 0.1430 - val_loss: 1.9460 - learning_rate: 0.0100
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.1459 - loss: 1.9461 - val_accuracy: 0.1430 - val_loss: 1.9460 - learning_rate: 1.0000e-03
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.1467 - loss: 1.9456 - val_accuracy: 0.1430 - val_loss: 1.9460 - learning_rate: 1.0000e-04
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.1467 - loss: 1.9460 - val_accuracy: 0.1430 - val_loss: 1.9460 - learning_rate: 1.0000e-05
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.1328 - loss: 1.9462 - val_accuracy: 0.1430 - val_loss: 1.9460 - learning_rate: 1.0000e-06
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.1425 - loss: 1.9459 - val_accu

In [ ]:
model.evaluate(test_generator)

##fine tune


In [ ]:
base_model.trainable =True
base_model.trainable

In [ ]:
for layer in base_model.layers[:-3]:
  layer.trainable = False

for layer in base_model.layers[-4:]:
  print(layer.name, layer.trainable)

In [ ]:
model_fine = create_resnet_model(256, 0.5)

In [ ]:
model_fine.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
current_time01 = datetime.datetime.now().strftime("%Y%m%d")
current_time02 = datetime.datetime.now().strftime("%H%M%S")

current_time = current_time01
model_name = 'resnet_fine_tune_01'

log_dir = f"logs/fit/{current_time}_{model_name}"

In [ ]:
# 모델 fitting
model_fine.fit(train_generator, batch_size=batch_size, validation_data=val_generator, epochs=10, callbacks = callbacks)

In [ ]:
# %tensorboard --logdir logs/fit